# 7장. Error Handling

- 응용 프로그램이 실행 중인 터미널 세션을 보면 오류 추적 가능
- 사용자에게는 최소한의 오류 정보를 제공(보안성 향상)
- 오류의 세부 정보는 서버 or 로그 파일에서 제공
- 모든 오류는 터미널에 저장

#### Debug Mode

- 응용 프로그램에 상세한 오류 정보 제공
- 아래와 같은 설정과정이 필요

```shell
$ set FLASK_ENV=development
```

- 디버그는 스택 프레임 확장하여 소스 코드 확인 가능
- 디버그는 상세한 오류 정보를 제공하기에 악의적인 접근이 가능
- 악의적인 접근을 막기 위해 block 상태로 시작 및 브러우저 로드 시 PIN 번호 요구
- 응용프로그램이 업데이트될 때마다 디버그에 자동 반영

#### Custom Error Pages

- 사용자 정의 오류 핸들러를 선언
- 데코레이터 사용
- 본 단원에선 404, 500 오류만 확인
  (500 오류는 데이터베이스 오류 후에 호출)

ex)
``` python
@app.errorhandler(404)
@app.errorhandler(500)
```

- 오류 정보를 제공하는 브라우저 __'404.html, 500.html'__ 작성 

#### Sending Error by Email

- 오류는 능동적인 관리가 중요
- 기본 디버그는 알림 기능을 제공하지 않음
- 오류 발생시 메일로 전송하는 기능 추가
- 단, Gmail 계정 보안 기능으로 오류 로그를 전송하지 못할 가능성 존재

```python
class Config(object):
    
    MAIL_SERVER = os.environ.get('MAIL_SERVER')
    MAIL_PORT = int(os.environ.get('MAIL_PORT') or 25)
    MAIL_USE_TLS = os.environ.get('MAIL_USE_TLS') is not None
    MAIL_USERNAME = os.environ.get('MAIL_USERNAME')
    MAIL_PASSWORD = os.environ.get('MAIL_PASSWORD')
    ADMINS = ['your-email@example.com']
```

- 이메일 세부 정보를 __config.py__ 파일에 추가
- python의 logging 패키지를 사용해 이메일로 오류 로그 보내는 기능 구현
- 아래 코드를 __init.py__ 파일에 추가함으로 오류 로그를 메일로 전송하는 기능을 간단하게 구현 가능 

```python
from logging.handlers import SMTPHandler
```

- 아래 코드를 사용해 터미널에서 이메일 로거를 활성화

```shell
$ python -m smtpd -n -c DebuggingServer localhost:8025
```

#### Logging to a File

- 응용 프로그램에 대한 로그 파일 유지 관리

```python
if not os.path.exists('logs'):
        os.mkdir('logs')
    file_handler = RotatingFileHandler('logs/microblog.log', maxBytes=10240,
                                       backupCount=10)
    file_handler.setFormatter(logging.Formatter(
        '%(asctime)s %(levelname)s: %(message)s [in %(pathname)s:%(lineno)d]'))
    file_handler.setLevel(logging.INFO)
    app.logger.addHandler(file_handler)

    app.logger.setLevel(logging.INFO)
    app.logger.info('Microblog startup')
```

- __microblog.log__ 를 log 디렉토리에 생성
- __RotatingFileHandler__ 클래스는 로그 파일의 크기가 너무 커지지 않도록 하는 클래스(10KB크기로 유지, 나머지 log 정보는 백업 파일에 보관)
- __logger.addHandler__ 클래스는 로그 메시지에 대한 사용자 지정 클래스 
  파일에 타임스탬프, 소스 파일, 로그 항목 등 다양한 정보를 담고 있음

#### Fixing the Duplicate Username Bug

- 사용자 이름이 중복저장되는 버그 해결
- 사용자 이름을 사용해도 되는지 유효성 검사

```python
    def validate_username(self, username):
        if username.data != self.original_username:
            user = User.query.filter_by(username=self.username.data).first()
            if user is not None:
                raise ValidationError('Please use a different username.')
```

- 사용자의 이름을 변수로 받아 현재 사용자의 이름과 같다면 사용자의 이름이 중복인지 확인할 필요 없음.
- 위와 같은 기능으로 함수를 사용하려면 사용자 이름을 인수로 추가